In [1]:
"""
[V7]
Blend 7 Models:
* kibuna-nn-hs-1024-last-train (aka. 2stage-NN, LB: 0.01822)
* simpleNN-oldcv (LB: 0.01836)
* deepinsight-efficientnet-v7-b3-infer (LB: 0.01850)
* deepinsight_resnest_lightning_v2_infer (LB: 0.01854)
* deepinsight_resnest_lightning_v1_infer (LB: 0.01853)
* improving-mark-s-2-heads-model-infer.py (LB: 0.1844)
* supervised-tabnet-lightning-v2-infer.py (LB: 0.1857)

Removed for now due to low weights:
* 503-203-tabnet-with-nonscored-features (LB: 0.01836)
* fork-of-2heads-looper-super-puper-markpeng-infer (LB: 0.1836)


"""

kernel_mode = False

import os
import numpy as np
import pandas as pd
import time
import random
import math
import datetime
import pickle
from pickle import dump, load
import glob

from numba import njit
from scipy.optimize import minimize, fsolve

import optuna

import warnings
warnings.filterwarnings('ignore')

import gc
gc.enable()

rand_seed = 1120

search_mode = True
run_submit_script = False

# method = "CV"
# method = "scipy_per_target"
# method = "scipy"
method = "optuna"
study_name = "blend_search_optuna_v7_10folds"
# study_name = "blend_search_optuna_v7"
# study_name = "blend_search_optuna_v6_per_target"
# n_trials = 500
n_trials = 3000
# n_trials = 5000
# n_trials = 10000

In [2]:
# !mkdir -p /root/.cache/torch/hub/checkpoints/
# !cp ../input/gen-efficientnet-pretrained/tf_efficientnet_*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v1-resnest50/*.pth /root/.cache/torch/hub/checkpoints/
# !cp ../input/deepinsight-resnest-v2-resnest50-output/*.pth /root/.cache/torch/hub/checkpoints/
# !ls -la /root/.cache/torch/hub/checkpoints/

In [3]:
# !cp ../input/kaggle-moa-team/scripts/* .
# !cp ../input/kaggle-moa-team/blends/*.pkl .
# !ls -la

In [4]:
dataset_folder = "../input/lish-moa" if kernel_mode else "/workspace/Kaggle/MoA/"

# Add your model inference script here
# Tuple Format: (script, oof_filename, output_filename, weight)
model_list = [
    # 10 folds
    ("2stageNN_with_ns_oldcv_10folds.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv_10folds.py",
     "../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv_10folds.npy",
     "submission_2stageNN_with_ns_oldcv_10folds.csv",
     0.36295047115403034),
    
    ("script_simpleNN_oldcv_10folds.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv_10folds.py",
     "../../Github/kaggle_moa_team/oof/oof_NN_oldcv_10fold.npy",
     "submission_script_simpleNN_oldcv_10folds.csv",
     0.00019665601658366993),
    
    #     ("2stageNN_with_ns_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py",
    #      "../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy",
    #      "submission_2stageNN_with_ns_oldcv_0.01822.csv",
    #      0.323528084383917),

    #     ("script_simpleNN_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py",
    #      "../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy",
    #      "submission_script_simpleNN_oldcv_0.01836.csv",
    #      0.08786476491118465),
    
    #     ("fork-of-2heads-looper-super-puper-markpeng-infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/fork-of-2heads-looper-super-puper-markpeng-infer.py",
    #      "../../Github/kaggle_moa_team/oof/oof_fork-of-2heads-looper-super-puper-markpeng.npy",
    #      "submission_2heads-looper-super-puper_0.01836.csv",
    #      0.018966959973949222),
    
    ("improving-mark-s-2-heads-model-infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py",
     "../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy",
     "submission_improving-mark-s-2-heads-model.csv",
     0.10383226438101152),
    
    ("deepinsight_efficientnet_lightning_v7_b3_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy",
     "submission_effnet_v7_b3_0.01850.csv",
     0.16907012564901389),
    
    ("deepinsight_resnest_lightning_v1_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v1_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy",
     "submission_resnest_v1_0.01853.csv",
     0.10211240990528128),
    
    ("deepinsight_resnest_lightning_v2_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py",
     "../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy",
     "submission_resnest_v2_0.01854.csv",
     0.26308884808653726),
    
    #     ("supervised_tabnet_v2_seeds_infer.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/supervised_tabnet_v2_seeds_infer.py",
    #      "../../Github/kaggle_moa_team/oof/oof_supervised_tabnet_v2_seeds_0.014939561499719265.npy",
    #      "submission_supervised_tabnet_v2_seeds_0.01857.csv",
    #      0),
    
    #     ("script_tabnet_ns_oldcv.py" if kernel_mode else "../../Github/kaggle_moa_team/scripts/script_tabnet_ns_oldcv.py",
    #      "../../Github/kaggle_moa_team/oof/oof_tabnet_ns_oldcv.npy",
    #      "submission_tabnet_ns_oldcv_0.01836.csv",
    #      0.0013224625996093413),
]

model_path = "." if kernel_mode else dataset_folder

In [5]:
train_features = pd.read_csv(f"{dataset_folder}/train_features.csv",
                             engine='c')
train_labels = pd.read_csv(f'{dataset_folder}/train_targets_scored.csv',
                           engine='c')
train_classes = [c for c in train_labels.columns if c != "sig_id"]

non_control_group_rows = train_features["cp_type"] == "trt_cp"
non_control_group_train_labels = train_labels.loc[
    non_control_group_rows, :].copy().reset_index(drop=True)

submission = pd.read_csv(f'{dataset_folder}/sample_submission.csv')
submission.iloc[:, 1:] = 0

In [6]:
def mean_logloss(y_pred, y_true):
    logloss = (1 - y_true) * np.log(1 - y_pred +
                                    1e-15) + y_true * np.log(y_pred + 1e-15)
    return np.nanmean(-logloss)


def save_pickle(obj, folder, name):
    dump(obj, open(f"{folder}/{name}.pkl", 'wb'), pickle.HIGHEST_PROTOCOL)


def load_pickle(path):
    return load(open(path, 'rb'))

In [7]:
# Reference: https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
# CPMP's logloss from https://www.kaggle.com/c/lish-moa/discussion/183010
def log_loss_numpy(y_pred, y_true):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, -np.log(y_pred),
                    -np.log(1 - y_pred))
    return loss.mean()

def func_numpy_metric(weights):
    oof_blend = np.tensordot(weights, all_oof, axes=((0), (0)))
    return log_loss_numpy(oof_blend, y_true)

@njit
def grad_func_jit(weights):
    oof_clip = np.minimum(1 - 1e-15, np.maximum(all_oof, 1e-15))
    gradients = np.zeros(all_oof.shape[0])
    for i in range(all_oof.shape[0]):
        a, b, c = y_true, oof_clip[i], np.zeros(
            (all_oof.shape[1], all_oof.shape[2]))
        for j in range(oof.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean(
            (-a * b + (b**2) * weights[i] + b * c) /
            ((b**2) *
             (weights[i]**2) + 2 * b * c * weights[i] - b * weights[i] +
             (c**2) - c))
    return gradients

## TPE (Tree-structured Parzen Estimator) and Sequential Least Squares Programming (SLSQP)
https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler

https://docs.scipy.org/doc/scipy/reference/optimize.minimize-slsqp.html

In [8]:
def run_inference_scripts(submission, weights=None, target_weights=None):
    for i, (script, oof_filename, output_filename, weight) in enumerate(model_list):
        print(f"Generating submission file from {script} ......")
        infer_start = time.time()
        !python {model_path}/{script}
        infer_elapsed = time.time() - infer_start
        print(f"Time spent on inference: {infer_elapsed/60:.2f} minutes.")

        model_submit = pd.read_csv(output_filename, engine='c')
        print(model_submit.head(5))
        print(model_submit.shape)

        if target_weights is not None:
            for j, target in enumerate(train_classes):
                print(f"Blending {script} for {target} with weight: {optimized_target_weights[j][i]} ......")
                submission.iloc[:, j+1] += model_submit.iloc[:, j+1] * optimized_target_weights[j][i]
        elif weights is None:
            print(f"Blending {script} with weight: {weight} ......")
            submission.iloc[:, 1:] += weight * model_submit.iloc[:, 1:]
        else:
            print(f"Blending {script} with weight: {weights[i]} ......")
            submission.iloc[:, 1:] += weights[i] * model_submit.iloc[:, 1:]

    return submission

In [9]:
total_start = time.time()

if not search_mode and run_submit_script:
    if method == "scipy_per_target":
        weights_path = glob.glob(f'{model_path}/{study_name}_*.pkl')[0]
        print(f"Loading target-wise optimized weights from {weights_path} ......")
        optimized_target_weights = load_pickle(weights_path)

        # For 206 target weights
        submission = run_inference_scripts(
            submission, target_weights=optimized_target_weights)
    else:
        submission = run_inference_scripts(submission)

elif search_mode and method == "CV":
    y_true = non_control_group_train_labels[train_classes].values

    all_oof = np.zeros(
        (len(model_list), non_control_group_train_labels.shape[0], 206))
    blend_oof = np.zeros((non_control_group_train_labels.shape[0], 206))
    print(all_oof.shape)
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        all_oof[i, :, :] = oof
        blend_oof += oof * weight

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")

    blend_oof_loss = mean_logloss(blend_oof, y_true)
    print(f"Blend OOF Validation Loss: {blend_oof_loss:.6f}\n")

elif search_mode and method == "optuna":
    print("[Optuna]")
    ## Search Best Blend Weights by Optuna ##
    model_oofs = []

    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        oof_loss = mean_logloss(
            oof, non_control_group_train_labels[train_classes].values)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    def objective(trial):
        weights = []
        for i in range(len(model_list)):
            weights.append(trial.suggest_float(f"w{i}", 0, 1.0))

        blend = np.zeros(model_oofs[0].shape)
        for i in range(len(model_list)):
            blend += weights[i] * model_oofs[i]
        blend = np.clip(blend, 0, 1.0)

        loss = mean_logloss(
            blend, non_control_group_train_labels[train_classes].values)
        return loss

    pruner = optuna.pruners.MedianPruner(
        n_startup_trials=5,
        n_warmup_steps=0,
        interval_steps=1,
    )
    sampler = optuna.samplers.TPESampler(seed=rand_seed)
    study = optuna.create_study(direction="minimize",
                                pruner=pruner,
                                sampler=sampler,
                                study_name=study_name,
                                storage=f'sqlite:///{study_name}.db',
                                load_if_exists=True)

    study.optimize(objective,
                   n_trials=n_trials,
                   timeout=None,
                   gc_after_trial=True,
                   n_jobs=-1)

    trial = study.best_trial

    if run_submit_script:
        optimal_weights = []
        for i, (script, oof_filename, output_filename,
                _) in enumerate(model_list):
            optimal_weights.append(trial.params[f"w{i}"])
        submission = run_inference_scripts(submission, weights=optimal_weights)

    print("\n[Optuna]")
    print("Number of finished trials: {}".format(len(study.trials)))
    print("Best trial:")
    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

elif search_mode and method == "scipy":
    print("[Scipy SLSQP]")
    # Optimise Blending Weights with Bonus
    # https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
    model_oofs = []
    y_true = non_control_group_train_labels[train_classes].values

    all_oof = np.zeros(
        (len(model_list), non_control_group_train_labels.shape[0], 206))
    print(all_oof.shape)
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        all_oof[i, :, :] = oof

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    tol = 1e-10
    init_guess = [1 / all_oof.shape[0]] * all_oof.shape[0]
    bnds = [(0, 1) for _ in range(all_oof.shape[0])]
    cons = {
        'type': 'eq',
        'fun': lambda x: np.sum(x) - 1,
        'jac': lambda x: [1] * len(x)
    }

    print('Inital Blend OOF:', func_numpy_metric(init_guess))

    start_time = time.time()
    res_scipy = minimize(
        fun=func_numpy_metric,
        x0=init_guess,
        method='SLSQP',
        # jac=grad_func_jit,  # grad_func
        bounds=bnds,
        constraints=cons,
        tol=tol)
    print("\n[Scipy SLSQP]")
    print(
        f'[{str(datetime.timedelta(seconds = time.time() - start_time))[2:7]}] Optimised Blend OOF:',
        res_scipy.fun)
    print(f'Optimised Weights: {res_scipy.x}\n')

    if run_submit_script:
        submission = run_inference_scripts(submission, weights=res_scipy.x)

# Target-wise Weight Optimization #

elif search_mode and method == "scipy_per_target":
    print("[Scipy SLSQP]")
    # Optimise Blending Weights with Bonus
    # https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0/notebook
    model_oofs = []
    y_true = non_control_group_train_labels[train_classes].values

    all_oof = np.zeros(
        (len(model_list), non_control_group_train_labels.shape[0], 206))
    print(all_oof.shape)
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        all_oof[i, :, :] = oof

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")
        model_oofs.append(oof)

    print("\n[Scipy SLSQP Per Target]")
    optimized_target_weights = []
    for i, target in enumerate(train_classes):
        tol = 1e-10
        init_guess = [1 / all_oof.shape[0]] * all_oof.shape[0]
        bnds = [(0, 1) for _ in range(all_oof.shape[0])]
        cons = {
            'type': 'eq',
            'fun': lambda x: np.sum(x) - 1,
            'jac': lambda x: [1] * len(x)
        }

        def func_numpy_metric_targes(weights):
            oof_blend = np.tensordot(weights,
                                     all_oof[:, :, i],
                                     axes=((0), (0)))
            return log_loss_numpy(oof_blend, y_true[:, i])

        start_time = time.time()
        res_scipy = minimize(
            fun=func_numpy_metric_targes,
            x0=init_guess,
            method='SLSQP',
            # jac=grad_func_jit,  # grad_func
            bounds=bnds,
            constraints=cons,
            tol=tol)

        print(
            f'[{str(datetime.timedelta(seconds = time.time() - start_time))[2:7]}] ' + \
            f'Optimised Blend OOF for {target}:', res_scipy.fun)
        print(f'Optimised Weights for {target}: {res_scipy.x}\n')
        optimized_target_weights.append(res_scipy.x)

    blend_targets_oof = np.zeros(
        (non_control_group_train_labels.shape[0], 206))
    for i, (script, oof_filename, output_filename,
            weight) in enumerate(model_list):
        print(f"Loading OOF from {oof_filename} ......")
        oof = np.load(f"{dataset_folder}/{oof_filename}")

        if oof.shape[0] == 23814:
            oof = oof[non_control_group_rows, :]

        for j in range(206):
            blend_targets_oof[:,
                              j] += oof[:, j] * optimized_target_weights[j][i]

        oof_loss = mean_logloss(oof, y_true)
        print(f"OOF Validation Loss of {script}: {oof_loss:.6f}\n")

    blend_targets_oof_loss = mean_logloss(blend_targets_oof, y_true)
    print(
        f"Blend Target-Wise OOF Validation Loss: {blend_targets_oof_loss:.6f}\n"
    )

    # Save optimized weights per target
    save_pickle(optimized_target_weights, model_path,
                f"{study_name}_{blend_targets_oof_loss}")

    if run_submit_script:
        # For 206 target weights
        submission = run_inference_scripts(
            submission, target_weights=optimized_target_weights)

[Optuna]
Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv_10folds.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv_10folds.py: 0.015461

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_NN_oldcv_10fold.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv_10folds.py: 0.015741

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
OOF Validation Loss of ../..

[I 2020-11-30 22:19:30,159] A new study created in RDB with name: blend_search_optuna_v7_10folds
[I 2020-11-30 22:19:30,899] Trial 5 finished with value: 0.02485472505747116 and parameters: {'w0': 0.9262256980905528, 'w1': 0.3657146659803088, 'w2': 0.6861408151183122, 'w3': 0.5037306133696665, 'w4': 0.6614756578530195, 'w5': 0.055298065475642644, 'w6': 0.0905610183409905}. Best is trial 5 with value: 0.02485472505747116.
[I 2020-11-30 22:19:31,130] Trial 2 finished with value: 0.02216511297817489 and parameters: {'w0': 0.38746762819316094, 'w1': 0.3302704599142642, 'w2': 0.29419916428851134, 'w3': 0.10610819465965793, 'w4': 0.42722785550820885, 'w5': 0.39785394535908847, 'w6': 0.9133522909105538}. Best is trial 2 with value: 0.02216511297817489.
[I 2020-11-30 22:19:31,147] Trial 0 finished with value: 0.023566324206767977 and parameters: {'w0': 0.5138977475093314, 'w1': 0.37250622310565984, 'w2': 0.4509369080496215, 'w3': 0.13620186301840154, 'w4': 0.34352721377030127, 'w5': 0.68775679

[I 2020-11-30 22:19:34,119] Trial 23 finished with value: 0.017420160492410134 and parameters: {'w0': 0.24230169027686088, 'w1': 0.25451082561112537, 'w2': 0.19328820170586541, 'w3': 0.32319844929969044, 'w4': 0.21276008492792597, 'w5': 0.39910978493689725, 'w6': 0.043674059918939856}. Best is trial 22 with value: 0.01732625736753544.
[I 2020-11-30 22:19:34,965] Trial 25 finished with value: 0.01815002410786728 and parameters: {'w0': 0.42724901277493865, 'w1': 0.006309915770665198, 'w2': 0.16624847084848965, 'w3': 0.4418541758720966, 'w4': 0.31389399577040145, 'w5': 0.33343918927768035, 'w6': 0.13275584806154878}. Best is trial 22 with value: 0.01732625736753544.
[I 2020-11-30 22:19:35,006] Trial 27 finished with value: 0.019546352216669432 and parameters: {'w0': 0.44285927299558614, 'w1': 0.26617282709102913, 'w2': 0.1847413722915974, 'w3': 0.4900813834485516, 'w4': 0.30018428676630743, 'w5': 0.33556003663245515, 'w6': 0.15833049972773114}. Best is trial 22 with value: 0.0173262573675

[I 2020-11-30 22:19:38,654] Trial 44 finished with value: 0.02011093837082374 and parameters: {'w0': 0.33412341665197687, 'w1': 0.054932422002176184, 'w2': 0.4063724433990681, 'w3': 0.4001154166270252, 'w4': 0.591547463199335, 'w5': 0.391721146674582, 'w6': 0.10047379481168801}. Best is trial 22 with value: 0.01732625736753544.
[I 2020-11-30 22:19:39,562] Trial 50 finished with value: 0.018064281715232732 and parameters: {'w0': 0.17661509738380296, 'w1': 0.45024774861259503, 'w2': 0.2765635663324329, 'w3': 0.059633938148905455, 'w4': 0.14508875071509472, 'w5': 0.24953901726714336, 'w6': 0.5414634400037562}. Best is trial 51 with value: 0.01703001646661964.
[I 2020-11-30 22:19:39,587] Trial 51 finished with value: 0.01703001646661964 and parameters: {'w0': 0.08082117601340982, 'w1': 0.37771570991838094, 'w2': 0.259723354671293, 'w3': 0.15956580909764462, 'w4': 0.110774143225827, 'w5': 0.15264168171171683, 'w6': 0.4771811816710355}. Best is trial 51 with value: 0.01703001646661964.
[I 20

[I 2020-11-30 22:19:43,224] Trial 71 finished with value: 0.016957766321046026 and parameters: {'w0': 0.04046155152222321, 'w1': 0.3634058931680223, 'w2': 0.055547241351883575, 'w3': 0.15678153257124045, 'w4': 0.07019956906660223, 'w5': 0.017517776311823333, 'w6': 0.8417934408096187}. Best is trial 61 with value: 0.016679473208779362.
[I 2020-11-30 22:19:43,813] Trial 74 finished with value: 0.017409844409076225 and parameters: {'w0': 0.0016344769347833021, 'w1': 0.46773527542818727, 'w2': 0.04411735737475844, 'w3': 0.1475330010147221, 'w4': 0.048793754179152785, 'w5': 0.07983347002346426, 'w6': 0.8894617723053627}. Best is trial 61 with value: 0.016679473208779362.
[I 2020-11-30 22:19:44,108] Trial 75 finished with value: 0.01741686869163742 and parameters: {'w0': 0.008789928231296672, 'w1': 0.43183231972015257, 'w2': 0.05407621310519356, 'w3': 0.14578940347379538, 'w4': 0.0469091801178366, 'w5': 0.08405277418728374, 'w6': 0.9071620212566596}. Best is trial 61 with value: 0.0166794732

...

[I 2020-11-30 22:29:47,565] Trial 2904 finished with value: 0.015137205520731152 and parameters: {'w0': 0.3553472871227471, 'w1': 0.0012582927591631944, 'w2': 0.08319847935610765, 'w3': 0.14331630031041678, 'w4': 0.17951281985129283, 'w5': 0.23670216394299892, 'w6': 0.028803622924874896}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:29:48,362] Trial 2908 finished with value: 0.015126066526033084 and parameters: {'w0': 0.3842456175473127, 'w1': 0.0009598219312832723, 'w2': 0.0454535583670116, 'w3': 0.11590766902139543, 'w4': 0.17561895304573838, 'w5': 0.2177720860307656, 'w6': 0.03055929011561421}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:29:48,715] Trial 2910 finished with value: 0.016618049682789984 and parameters: {'w0': 0.3132272187148868, 'w1': 0.0005783792781939023, 'w2': 0.043180552759664495, 'w3': 0.1624141331951698, 'w4': 0.24835472377422105, 'w5': 0.6249718713413833, 'w6': 0.03044940767464168}. Best is trial 2563 with value

[I 2020-11-30 22:29:54,287] Trial 2927 finished with value: 0.015122495539014012 and parameters: {'w0': 0.37665971533178205, 'w1': 0.0003604967646428771, 'w2': 0.11754321340745873, 'w3': 0.10133486863099023, 'w4': 0.14638063390303727, 'w5': 0.24470112349954645, 'w6': 0.018004041125138915}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:29:55,461] Trial 2929 finished with value: 0.01603858324418917 and parameters: {'w0': 0.3126198122481208, 'w1': 0.0002339115748746512, 'w2': 0.13384552027420168, 'w3': 0.13636051807981203, 'w4': 0.1568554006627514, 'w5': 0.22010028614903357, 'w6': 0.3720682427720625}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:29:55,860] Trial 2934 finished with value: 0.015344868257761286 and parameters: {'w0': 0.31580964387241606, 'w1': 0.0008477656141719671, 'w2': 0.10477019746532333, 'w3': 0.1486346188117609, 'w4': 0.1325388876617605, 'w5': 0.05325016279819461, 'w6': 0.0005946861113768273}. Best is trial 2563 with val

[I 2020-11-30 22:30:01,330] Trial 2952 finished with value: 0.015557764332728513 and parameters: {'w0': 0.3697396874439476, 'w1': 0.054317512182827356, 'w2': 0.13614192639122652, 'w3': 0.14470285371933214, 'w4': 0.1842634268455134, 'w5': 0.25765544919336686, 'w6': 0.018350855012839666}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:30:01,336] Trial 2948 finished with value: 0.015608672840920683 and parameters: {'w0': 0.390638338128108, 'w1': 0.056636228950368676, 'w2': 0.1358979350982995, 'w3': 0.14571894518409784, 'w4': 0.17713034848569992, 'w5': 0.25929436734795014, 'w6': 0.019894333230809396}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:30:03,012] Trial 2957 finished with value: 0.016068932020628462 and parameters: {'w0': 0.39017957325422875, 'w1': 0.02389768226836666, 'w2': 0.41130171610771993, 'w3': 0.1606602487437578, 'w4': 0.15040411534469286, 'w5': 0.17169869672685714, 'w6': 0.0005601996512778529}. Best is trial 2563 with value:

[I 2020-11-30 22:30:07,798] Trial 2977 finished with value: 0.015160254211276852 and parameters: {'w0': 0.3717664935273821, 'w1': 0.0007014691879639596, 'w2': 0.06854770424826914, 'w3': 0.14676532166233008, 'w4': 0.14924828615783164, 'w5': 0.2672673101094805, 'w6': 0.0334408954957307}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:30:07,803] Trial 2976 finished with value: 0.01798806784636499 and parameters: {'w0': 0.36400319218413113, 'w1': 0.0008609592983788686, 'w2': 0.14505913733255893, 'w3': 0.7385203028360763, 'w4': 0.1441688304005813, 'w5': 0.2766201149849515, 'w6': 0.00015572050627886633}. Best is trial 2563 with value: 0.015109516210552868.
[I 2020-11-30 22:30:07,809] Trial 2978 finished with value: 0.015420532224723862 and parameters: {'w0': 0.3711337176440949, 'w1': 0.0323307883751555, 'w2': 0.14806224568970963, 'w3': 0.1488641323655741, 'w4': 0.14804992463286104, 'w5': 0.27467989826496186, 'w6': 0.0007854436147872911}. Best is trial 2563 with value: 


[Optuna]
Number of finished trials: 3000
Best trial:
  Value: 0.015109516210552868
  Params: 
    w0: 0.345350187040706
    w1: 0.018804585817609087
    w2: 0.09890095339659595
    w3: 0.15486138107425962
    w4: 0.1302653777195202
    w5: 0.24674004919729609
    w6: 0.00020099415280697705


In [10]:
total_elapsed = time.time() - total_start
print(f"Total time spent: {total_elapsed/60:.2f} minutes.")

Total time spent: 10.74 minutes.


In [11]:
# [V7 - Improved 2heads, Mark's TabNet V2 Seeds]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015109516210552868
#   Params: 
#     w0: 0.345350187040706
#     w1: 0.018804585817609087
#     w2: 0.09890095339659595
#     w3: 0.15486138107425962
#     w4: 0.1302653777195202
#     w5: 0.24674004919729609
#     w6: 0.00020099415280697705

# [Scipy SLSQP]
# [00:32] Optimised Blend OOF: 0.015107023232510362
# Optimised Weights: [0.36394211 0.         0.08646557 0.17174434 0.1059386  0.27190938
#  0.        ]

In [12]:
# [V7 - without TabNet, Mark's 2heads, 10-folds 2StageNN and SimpleNN, ResNeSt V1]

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv_10folds.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv_10folds.py: 0.015461

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_NN_oldcv_10fold.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv_10folds.py: 0.015741

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756

# [Optuna]
# Number of finished trials: 5000
# Best trial:
#   Value: 0.015118051559985067
#   Params: 
#     w0: 0.3693123925185777
#     w1: 0.0004358344602283515
#     w2: 0.11986363820431883
#     w3: 0.18219399304034917
#     w4: 0.32927717286142916

# [Scipy SLSQP]
# [00:24] Optimised Blend OOF: 0.015117984713726062
# Optimised Weights: [3.79183409e-01 8.13151629e-19 1.08023618e-01 1.86335277e-01
#  3.26457696e-01]

In [13]:
# [V7 - without TabNet, Mark's 2heads, 10-folds 2StageNN and SimpleNN]
# [Optuna]
# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv_10folds.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv_10folds.py: 0.015461

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_NN_oldcv_10fold.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py: 0.015741

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v1_infer.py: 0.015819

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy ......

# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756

# [Optuna]
# Number of finished trials: 10000
# Best trial:
#   Value: 0.015107277135046551
#   Params: 
#     w0: 0.36295047115403034
#     w1: 0.00019665601658366993
#     w2: 0.10383226438101152
#     w3: 0.16907012564901389
#     w4: 0.10211240990528128
#     w5: 0.26308884808653726

# [Scipy SLSQP]
# [00:48] Optimised Blend OOF: 0.015107004381942337
# Optimised Weights: [0.36727479 0.         0.08130163 0.17091322 0.10863643 0.27187394]

In [14]:
# [V7 - without TabNet, Mark's 2heads]
# [Optuna]
# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py: 0.015606

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py: 0.015846

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v1_infer.py: 0.015819

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756

# [Optuna]
# Number of finished trials: 10000
# Best trial:
#   Value: 0.015146055466812214
#   Params: 
#     w0: 0.2590485239176548
#     w1: 0.033637086725830254
#     w2: 0.12194872432858535
#     w3: 0.19075874431151257
#     w4: 0.11084230101870868
#     w5: 0.28295491196215444

# [Scipy SLSQP]
# [00:30] Optimised Blend OOF: 0.01514578929579469
# Optimised Weights: [0.25256016 0.03296526 0.13166163 0.18166774 0.11849445 0.28265076]

In [15]:
# [V7 - without TabNet]
# [Optuna]
# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_2stageNN_ns_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/2stageNN_with_ns_oldcv.py: 0.015606

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_script_simpleNN_oldcv.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/script_simpleNN_oldcv.py: 0.015846

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_fork-of-2heads-looper-super-puper-markpeng.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/fork-of-2heads-looper-super-puper-markpeng-infer.py: 0.015887

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_improving-mark-s-2-heads-model_0.015660083675738144.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/improving-mark-s-2-heads-model-infer.py: 0.015660

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_efficientnet_lightning_v7_b3_0.01850.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_efficientnet_lightning_v7_b3_infer.py: 0.016016

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v1_resnest50_0.014619621213185928.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v1_infer.py: 0.015819

# Loading OOF from ../../Github/kaggle_moa_team/oof/oof_deepinsight_ResNeSt_v2_resnest50_0.01854.npy ......
# OOF Validation Loss of ../../Github/kaggle_moa_team/scripts/deepinsight_resnest_lightning_v2_infer.py: 0.015756

# Number of finished trials: 10000
# Best trial:
#   Value: 0.015146827058312776
#   Params: 
#     w0: 0.2507759947969869
#     w1: 0.03339376719044784
#     w2: 8.528382280836113e-05
#     w3: 0.11625864567289637
#     w4: 0.18770116032490702
#     w5: 0.09530145379465743
#     w6: 0.3164264877487286

# [Scipy SLSQP]
# [00:45] Optimised Blend OOF: 0.015145788791365362
# Optimised Weights: [2.52672054e-01 3.32734786e-02 1.05430191e-17 1.30957268e-01
#  1.82060938e-01 1.18387855e-01 2.82648406e-01]

In [16]:
# [V6 - without TabNet, 2heads]
# Total time spent: 0.68 minutes.
# Blend Target-Wise OOF Validation Loss: 0.015044

In [17]:
# [V6 - without TabNet, 2heads]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015171999561900233
#   Params: 
#     w0: 0.323528084383917
#     w1: 0.08786476491118465
#     w2: 0.21849845883367852
#     w3: 0.3704230222796271

# [Scipy SLSQP]
# [00:22] Optimised Blend OOF: 0.015172004593585666
# Optimised Weights: [0.32020133 0.09043987 0.22122948 0.36812932]

In [18]:
# [V6 - without TabNet]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015172424601530761
#   Params: 
#     w0: 0.3138176484100186
#     w1: 0.07850519440561339
#     w2: 0.0007183363099561991
#     w3: 0.23849563017967007
#     w4: 0.3694870328388392

# [Scipy SLSQP]
# [00:21] Optimised Blend OOF: 0.015172004898867827
# Optimised Weights: [0.32045559 0.09026525 0.         0.22069638 0.36858278]

In [19]:
# [V6]
# [Optuna]
# Number of finished trials: 5000
# Best trial:
#   Value: 0.015173437622007157
#   Params: 
#     w0: 0.30923325055652684
#     w1: 0.09831493504786226
#     w2: 0.018966959973949222
#     w3: 0.19863369862866234
#     w4: 0.0013224625996093413
#     w5: 0.3728865483320761

# [Scipy SLSQP]
# [00:36] Optimised Blend OOF: 0.015172005464591968
# Optimised Weights: [3.20472642e-01 9.01191588e-02 1.78893358e-18 2.20448482e-01
#  3.27971157e-18 3.68959717e-01]

In [20]:
# [V5]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015344701181290615
#   Params: 
#     w0: 0.5141433844379889
#     w1: 0.11747776562133813
#     w2: 0.3668324643717302

# [00:14] Optimised Blend OOF: 0.015344695215068541
# Optimised Weights: [0.51922623 0.11292509 0.36784869]

In [21]:
# [V4]
# [Optuna]
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015331901615194453
#   Params: 
#     w0: 0.4505928450756189
#     w1: 0.13010257032841785
#     w2: 0.06308933354044946
#     w3: 0.35639153615958885
#
# [Scipy]
# [00:23] Optimised Blend OOF: 0.015331777381591449
# Optimised Weights: [0.44090106 0.14508641 0.05945655 0.35455598]

In [22]:
# [V3]
# improving-mark-s-2-heads-model-infer
# Number of finished trials: 3000
# Best trial:
#   Value: 0.01515466145873492
#   Params: 
#     w0: 0.0002980690037490555
#     w1: 0.29771381784976886
#     w2: 0.1569191862042946
#     w3: 0.18156875605872544
#     w4: 0.36371774630338105

In [23]:
# [V3]
# fork-of-2heads-looper-super-puper-markpeng-infer
# Number of finished trials: 3000
# Best trial:
#   Value: 0.015170138066049686
#   Params: 
#     w0: 0.00019903389488299251
#     w1: 0.3853752127955825
#     w2: 0.015968332256452233
#     w3: 0.22945916769823432
#     w4: 0.3711290150522236

In [24]:
if search_mode and method == "scipy_per_target":
    # OOF scores per target
    target_oof_losses = []
    for i, target in enumerate(train_classes):
        print(target)
        # print(y_true[:, i])

        oof_loss = mean_logloss(blend_targets_oof[:, i], y_true[:, i])
        target_oof_losses.append(oof_loss)
        print(f"Blend OOF Validation Loss of {target}: {oof_loss:.6f}\n")

    target_loss_df = pd.DataFrame(
        data={
            "target": train_classes,
            "oof_logloss": target_oof_losses
        },
        columns=["target", "oof_logloss"
                 ]).sort_values(by="oof_logloss",
                                ascending=False).reset_index(drop=True)
    print(target_loss_df)

In [25]:
if run_submit_script:
    print(submission.shape)
    print(submission)
    submission.to_csv('submission.csv', index=False)

## EOF

In [26]:
if kernel_mode:
    !rm ./*.py
    !ls -la